In [1]:
from math import radians, sin, cos, sqrt, asin
from pymongo import ASCENDING
import pandas as pd
import json
from shapely.geometry import LineString
from datetime import datetime, timedelta
import urllib.parse
from pymongo.mongo_client import MongoClient
from pymongo import ASCENDING
from pymongo.server_api import ServerApi
import pandas as pd
import geopandas as gpd

# MongoDB Atlas Configuration
username = urllib.parse.quote_plus("root")
password = urllib.parse.quote_plus("root")
cluster_url = "cluster0.dzcyoux.mongodb.net"

# MongoDB URI with additional connection parameters for better performance
uri = f"mongodb+srv://{username}:{password}@{cluster_url}/?retryWrites=true&w=majority"

client = MongoClient(uri, server_api=ServerApi('1'))
db = client["gtfs"]



In [ ]:

stops_df = pd.DataFrame(list(db["cagliari_ctm_stops"].find()))

lat_start, lon_start = 9.113601807007171, 39.217706273420724  # piazza yenne
lat_end, lon_end = 9.160817004740238, 39.2016934400977  # poetto
orario = "16:30:00"

origin = (lat_start, lon_start)  # (lat, lon)
destination = (lat_end, lon_end)  # (lat, lon)

def haversine_ref_point(row, lat, lon):
    lon1 = lon
    lat1 = lat
    lon2 = row['stop_lon']
    lat2 = row['stop_lat']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km


def get_stop_destinazione(lat, lon, stops_df):
    stop_destinazione = stops_df[["stop_id", "stop_name", "stop_lat", "stop_lon"]].copy()

    # calcolo la distanza tra dove voglio andare e le fermate, cerco quella più vicina
    stop_destinazione["distance"] = stop_destinazione.apply(lambda row: haversine_ref_point(row, lat, lon), axis=1)
    stop_destinazione = stop_destinazione[stop_destinazione["distance"] == stop_destinazione["distance"].min()]
    return stop_destinazione.iloc[0]


stops_df = pd.DataFrame(list(db["cagliari_ctm_stops"].find()))
orario = "16:30:00"

# trovo lo stop più vicino alla destinazione
stop_destinazione_df = get_stop_destinazione(destination[1], destination[0], stops_df)

# prendo in considerazione l'orario
orario = datetime.strptime(orario, "%H:%M:%S")

#considero l'intervallo tra il mio arrivo e un ora prima
orario_earlier = (orario - timedelta(hours=1)).strftime("%H:%M:%S")
orario = orario.strftime("%H:%M:%S")  # Convert back to string!

# trovo i trip che comprendono la fermata della destinazione, orario di arrivo più vicina al mio orario
destinazione = stop_destinazione_df["stop_id"]
closest_trip_doc = db["cagliari_ctm_stop_times"].find(
    {
        "stop_id": destinazione,
        "arrival_time": {"$gte": orario_earlier, "$lte": orario}
    },
    {
        "_id": 0,
        "trip_id": 1,
        "arrival_time": 1
    }
).sort("arrival_time", -1)  # descending order to get closest before or at orario

# to_df
closest_trip_df = pd.DataFrame(list(closest_trip_doc))

# stop times dei trip precedenti
trip_ids = closest_trip_df["trip_id"].unique().tolist()
stop_times_cursor = db["cagliari_ctm_stop_times"].find(
    {
        "trip_id": {"$in": trip_ids}},
    {
        "_id": 0,
        "trip_id": 1,
        "arrival_time": 1,
        "departure_time": 1,
        "stop_id": 1,
        "stop_sequence": 1
    }
).sort([
    ("trip_id", ASCENDING),
    ("stop_sequence", ASCENDING)
])
df_stop_times_filtered = pd.DataFrame(list(stop_times_cursor))

# trovare di quei trip quale abbia la fermata più vicina al mio punto di partenza
stops_viaggio_gr = df_stop_times_filtered[["stop_id", "trip_id","arrival_time", "stop_sequence"]].merge(stops_df[["stop_id", "stop_name", "stop_lat", "stop_lon"]], on="stop_id")
stop_partenza_df = get_stop_destinazione(origin[1], origin[0], stops_viaggio_gr)
partenza = stop_partenza_df["stop_id"]

# trip viaggio finale
trip_viaggio = stops_viaggio_gr[stops_viaggio_gr["stop_id"] == partenza].sort_values("arrival_time", ascending=False).iloc[0]["trip_id"]

# stop times viaggio finale
df_stop_times_viaggio = stops_viaggio_gr[stops_viaggio_gr["trip_id"] == trip_viaggio]

In [ ]:
index_partenza = df_stop_times_viaggio[df_stop_times_viaggio["stop_id"] == partenza].index[0]
index_destinazione = df_stop_times_viaggio[df_stop_times_viaggio["stop_id"] == destinazione].index[0]
df_stop_times_viaggio = df_stop_times_viaggio.loc[index_partenza:index_destinazione]
df_stop_times_viaggio

In [ ]:
trips_cursor = db["cagliari_ctm_trips"].find(
        {
            "trip_id": int(trip_viaggio)  # Direct comparison instead of $in
        },
        {
            "_id": 0,
            "route_id": 1,
            "service_id": 1,
            "trip_id": 1,
            "trip_headsign": 1,
            "direction_id": 1,
            "shape_id": 1
        }
    )

df_trips_filtered = pd.DataFrame(list(trips_cursor))
df_trips_filtered

In [ ]:
shape_id = df_trips_filtered.iloc[0]["shape_id"]

# trovare tutte le shape del percorso
shapes_cursor = db["cagliari_ctm_shapes"].find(
    {
        "shape_id": shape_id  # Direct comparison instead of $in
    },
    {
        "_id": 0,
        "shape_id": 1,
        "shape_pt_lat": 1,
        "shape_pt_lon": 1,
        "shape_pt_sequence": 1
    }
).sort([("shape_pt_sequence", ASCENDING)])

df_shapes_filtered = pd.DataFrame(list(shapes_cursor))
df_shapes_filtered

In [ ]:
lat_partenza = df_stop_times_viaggio.iloc[0]["stop_lat"].round(3)
lon_partenza = df_stop_times_viaggio.iloc[0]["stop_lon"].round(3)
shape_sequence_partenza = df_shapes_filtered[(df_shapes_filtered["shape_pt_lat"].round(3) == lat_partenza) & 
                   (df_shapes_filtered["shape_pt_lon"].round(3) == lon_partenza)].iloc[0]["shape_pt_sequence"]

lat_destinazione = df_stop_times_viaggio.iloc[-1]["stop_lat"].round(3)
lon_destinazione = df_stop_times_viaggio.iloc[-1]["stop_lon"].round(3)
shape_sequence_destinazione = df_shapes_filtered[(df_shapes_filtered["shape_pt_lat"].round(3) == lat_destinazione) & 
                   (df_shapes_filtered["shape_pt_lon"].round(3) == lon_destinazione)].iloc[0]["shape_pt_sequence"]


In [ ]:
index_shape_partenza = df_shapes_filtered[df_shapes_filtered["shape_pt_sequence"] == shape_sequence_partenza].index[0]
index_shape_destinazione = df_shapes_filtered[df_shapes_filtered["shape_pt_sequence"] == shape_sequence_destinazione].index[0]
df_shapes_filtered = df_shapes_filtered.loc[index_shape_partenza:index_shape_destinazione]
df_shapes_filtered

### analisi visualizzazione delle shape per linea

In [4]:
df_shapes = pd.DataFrame(list(db["cagliari_ctm_shapes"].find()))
df_shapes

,_id,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,6891e889782a7bf4254a25d9,00CA04,39.23523,9.13131,3
1,6891e889782a7bf4254a25da,00CA04,39.23528,9.13129,4
2,6891e889782a7bf4254a25e1,00CA04,39.23577,9.13145,11
3,6891e889782a7bf4254a25fa,00CA04,39.23473,9.13092,36
4,6891e889782a7bf4254a260f,00CA04,39.23618,9.13240,57
...,...,...,...,...,...
62536,6891e889782a7bf4254b1a05,U_ER02,39.23769,9.10027,401
62537,6891e889782a7bf4254b1a07,U_ER02,39.23758,9.10033,403
62538,6891e889782a7bf4254b1a01,U_ER02,39.23809,9.09977,397
62539,6891e889782a7bf4254b1a13,U_ER02,39.23797,9.10191,415


In [ ]:
gdf = gpd.GeoDataFrame(
            df_shapes[["shape_pt_lon", "shape_pt_lat"]],
            geometry=gpd.points_from_xy(
                df_shapes["shape_pt_lon"], 
                df_shapes["shape_pt_lat"]
            ), 
            crs="EPSG:4326"
        )
gdf

,shape_pt_lon,shape_pt_lat,geometry
0,9.13131,39.23523,POINT (9.13131 39.23523)
1,9.13129,39.23528,POINT (9.13129 39.23528)
2,9.13145,39.23577,POINT (9.13145 39.23577)
3,9.13092,39.23473,POINT (9.13092 39.23473)
4,9.13240,39.23618,POINT (9.13240 39.23618)
...,...,...,...
62536,9.10027,39.23769,POINT (9.10027 39.23769)
62537,9.10033,39.23758,POINT (9.10033 39.23758)
62538,9.09977,39.23809,POINT (9.09977 39.23809)
62539,9.10191,39.23797,POINT (9.10191 39.23797)


In [12]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPoint


def get_coordinate_bounds(df, lat_col='shape_pt_lat', lon_col='shape_pt_lon'):
    """
    Calculate the convex hull (outer bounds) of coordinates in a DataFrame
    
    Args:
        df: pandas DataFrame with coordinate data
        lat_col: name of latitude column
        lon_col: name of longitude column
    
    Returns:
        GeoDataFrame with convex hull geometry and bounds dictionary
    """
    # Create GeoDataFrame from coordinates
    gdf = gpd.GeoDataFrame(
                df[["shape_pt_lon", "shape_pt_lat"]],
                geometry=gpd.points_from_xy(
                    df[lon_col], 
                    df[lat_col]
                ), 
                crs="EPSG:4326"
            )
    
    # Calculate convex hull using union_all() instead of unary_union
    if len(gdf) > 0:
        # Use union_all() to combine all geometries
        combined_geometry = MultiPoint(gdf.geometry.tolist())
        
        # Calculate convex hull
        convex_hull = combined_geometry.convex_hull
        
        # Get bounds (minx, miny, maxx, maxy)
        bounds = convex_hull.bounds
        
        # Create result dictionary
        bounds_dict = {
            'min_lon': bounds[0],
            'min_lat': bounds[1],
            'max_lon': bounds[2],
            'max_lat': bounds[3],
            'center_lon': (bounds[0] + bounds[2]) / 2,
            'center_lat': (bounds[1] + bounds[3]) / 2
        }
        
        # Create GeoDataFrame for the hull
        hull_gdf = gpd.GeoDataFrame(
            {'geometry': [convex_hull]}, 
            crs='EPSG:4326'
        )
        
        return hull_gdf, bounds_dict
    else:
        # Return empty results if no data
        empty_hull = gpd.GeoDataFrame({'geometry': []}, crs='EPSG:4326')
        empty_bounds = {
            'min_lon': 0, 'min_lat': 0, 'max_lon': 0, 'max_lat': 0,
            'center_lon': 0, 'center_lat': 0
        }
        return empty_hull, empty_bounds

In [23]:
hull_gdf, bounds = get_coordinate_bounds(df_shapes)
hull_gdf

,geometry
0,"POLYGON ((9.15167 39.18684, 9.08415 39.20808, ..."


In [25]:
hull_geojson = json.loads(hull_gdf.to_json())
hull_geojson


{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[9.15167, 39.18684],
      [9.08415, 39.20808],
      [9.08039, 39.21041],
      [8.96148, 39.30799],
      [8.96141, 39.30806],
      [8.96136, 39.30812],
      [8.96127, 39.30825],
      [8.96072, 39.30915],
      [8.96071, 39.30918],
      [8.96071, 39.30922],
      [8.96072, 39.30925],
      [8.96073, 39.30926],
      [8.96074, 39.30927],
      [8.96077, 39.30929],
      [8.96081, 39.30931],
      [8.96085, 39.30932],
      [8.96364, 39.30992],
      [8.97419, 39.31163],
      [9.27323, 39.24453],
      [9.27348, 39.24446],
      [9.27358, 39.24443],
      [9.27396, 39.24431],
      [9.27417, 39.24423],
      [9.27441, 39.24413],
      [9.27453, 39.24405],
      [9.27463, 39.24398],
      [9.33306, 39.19989],
      [9.33395, 39.19902],
      [9.33402, 39.19894],
      [9.33406, 39.19887],
      [9.33409, 39.19876],
      [9.33409

In [20]:
from maplibre.ipywidget import MapWidget as Map
from maplibre.layer import Layer, LayerType

hull_geojson = json.loads(hull_gdf.to_json())

m = Map(
    style="https://demotiles.maplibre.org/style.json",
    center=[bounds['center_lon'], bounds['center_lat']],
    zoom=12
)

m.set_data(source_id="hull", data=hull_geojson)

# Add hull layers
hull_layer = Layer(
    id="hull-layer",
    type=LayerType.FILL,
    source="hull",
    paint={
        "line-color": "#007cbf",
        "line-width": 3
    }
)

m.add_layer(hull_layer)
m

MapWidget(height='400px', map_options={'style': 'https://demotiles.maplibre.org/style.json', 'center': [9.1474…

In [22]:
from maplibre.ipywidget import MapWidget as Map

m = Map()
m

MapWidget(height='400px', map_options={'style': 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.j…